In [ ]:
# %pip install gym keras tensorflow numpy

# 📚 Import libraries
import gym
import numpy as np
import random
from collections import deque
from tensorflow import keras
from tensorflow.keras import layers

# 1. Create a Gym Environment
env = gym.make('CartPole-v1')  # Example game: CartPole

# 2. Define a Random Agent (Optional if you want to try random playing first)
class RandomAgent:
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()

# Instantiate the Random Agent
random_agent = RandomAgent(env.action_space)

# Try Random Agent
episodes = 5
for episode in range(episodes):
    observation, _ = env.reset()  # FIXED
    done = False
    total_reward = 0

    while not done:
        action = random_agent.act(observation)
        observation, reward, terminated, truncated, info = env.step(action)  # FIXED
        done = terminated or truncated
        total_reward += reward
        env.render()

    print(f"Random Agent - Episode {episode + 1}: Total Reward = {total_reward}")

env.close()

# 3. Define DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0    # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = keras.Sequential([
            layers.Dense(64, input_dim=self.state_size, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(np.expand_dims(state, axis=0), verbose=0)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(np.expand_dims(next_state, axis=0), verbose=0)[0])
            target_f = self.model.predict(np.expand_dims(state, axis=0), verbose=0)
            target_f[0][action] = target
            self.model.fit(np.expand_dims(state, axis=0), target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# 4. Instantiate DQN Agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

# 5. Train the DQN Agent
episodes = 100
batch_size = 32

for e in range(episodes):
    state, _ = env.reset()  # FIXED
    done = False
    total_reward = 0

    while not done:
        action = agent.act(state)
        next_state, reward, terminated, truncated, info = env.step(action)  # FIXED
        done = terminated or truncated
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            print(f"DQN Agent - Episode {e+1}/{episodes}, Total Reward: {total_reward}")

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

env.close()


Random Agent - Episode 1: Total Reward = 33.0
Random Agent - Episode 2: Total Reward = 23.0
Random Agent - Episode 3: Total Reward = 25.0
Random Agent - Episode 4: Total Reward = 21.0
Random Agent - Episode 5: Total Reward = 46.0
DQN Agent - Episode 1/100, Total Reward: 26.0
DQN Agent - Episode 2/100, Total Reward: 15.0
DQN Agent - Episode 3/100, Total Reward: 30.0
DQN Agent - Episode 4/100, Total Reward: 14.0
DQN Agent - Episode 5/100, Total Reward: 44.0
DQN Agent - Episode 6/100, Total Reward: 19.0
DQN Agent - Episode 7/100, Total Reward: 18.0
DQN Agent - Episode 8/100, Total Reward: 16.0
DQN Agent - Episode 9/100, Total Reward: 29.0
DQN Agent - Episode 10/100, Total Reward: 15.0
DQN Agent - Episode 11/100, Total Reward: 23.0
DQN Agent - Episode 12/100, Total Reward: 22.0
DQN Agent - Episode 13/100, Total Reward: 26.0
DQN Agent - Episode 14/100, Total Reward: 17.0
DQN Agent - Episode 15/100, Total Reward: 23.0
DQN Agent - Episode 16/100, Total Reward: 33.0
DQN Agent - Episode 17/100,